# Lending Club - Predicting Loan Payoff Timeliness
## Dana Hagist

### Introduction to project and Lending Club

In this course, we will walk through the full data science life cycle, from data cleaning and feature selection to machine learning. We will focus on credit modelling, a well known data science problem that focuses on modeling a borrower's credit risk (https://en.wikipedia.org/wiki/Credit_risk). Credit has played a key role in the economy for centuries and some form of credit has existed since the beginning of commerce. We'll be working with financial lending data from Lending Club (https://www.lendingclub.com/). Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. You can read more about their marketplace here (https://www.lendingclub.com/public/how-peer-lending-works.action).

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using past historical data (and their own data science process!) and assign an interest rate to the borrower. The interest rate is the percent in addition to the requested loan amount the borrower has to pay back. You can read more about the interest rate that Lending Club assigns here (https://www.lendingclub.com/public/borrower-rates-and-fees.action). Lending Club also tries to verify each piece of information the borrower provides but it can't always verify all of the information (usually for regulation reasons).

A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a grade (https://www.lendingclub.com/public/rates-and-fees.action) according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Investors are primarily interested in receiveing a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the origination fee (https://help.lendingclub.com/hc/en-us/articles/214501207-What-is-the-origination-fee-)that Lending Club charges.

The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. This means that investors don't have to wait until the full amount is paid off to start to see money back. If a loan is fully paid off on time, the investors make a return which corresponds to the interest rate the borrower had to pay in addition the requested amount. Many loans aren't completely paid off on time, however, and some borrowers default (https://www.lendingclub.com/public/collections-process.action) on the loan.

While Lending Club has to be extremely savvy and rigorous with their credit modelling, investors on Lending Club need to be equally as savvy about determining which loans are more likely to be paid off. While at first, you may wonder why investors would put money into anything but low interest loans. The incentive investors have to back higher interest loans is, well, the higher interest! If investors believe the borrower can pay back the loan, even if he or she has a weak financial history, then investors can make more money through the larger additional amount the borrower has to pay.

Most investors use a portfolio strategy to invest small amounts in many loans, with healthy mixes of low, medium, and interest loans. In this course, we'll focus on the mindset of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time. To do that, we'll need to first understand the features in the dataset and then experiment with building machine learning models that reliably predict if a loan will be paid off or not.

### Introducing the Data

Lending Club releases data for all of the approved and declined loan applications periodically on their website at https://www.lendingclub.com/info/download-data.action. You can select a few different year ranges to download the datasets (in CSV format) for both approved and declined loans.

You'll also find a data dictionary (in XLS format) which contains information on the different column names towards the bottom of the page. We recommend downloading the data dictionary to so you can refer to it whenever you want to learn more about what a column represents in the datasets. Here's a link to the data dictionary file hosted on Google Drive (https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit).

Before diving into the datasets themselves, let's get familiar with the data dictionary. The LoanStats sheet describes the approved loans datasets and the RejectStats describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on data on approved loans only.

The approved loans datasets contain information on current loans, completed loans, and defaulted loans. Let's now define the problem statement for this machine learning project:

- Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

Before we can start doing machine learning, we need to define what features we want to use and which column repesents the target column we want to predict. Let's start by reading in the dataset and exploring it.

### Dataquest Data Prep

In this mission, we'll focus on approved loans data from 2007 to 2011, since a good number of the loans have already finished. In the datasets for later years, many of the loans are current and still being paid off.

To ensure that code runs fast on our platform, we reduced the size of LoanStats3a.csv by:

- removing the first line:
    - because it contains the extraneous text Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action) instead of the column titles, which prevents the dataset from being parsed by the pandas library properly
- removing the desc column:
    - which contains a long text explanation for each loan
- removing the url column:
    - which contains a link to each loan on Lending Club which can only be accessed with an investor account
- removing all columns containing more than 50% missing values:
    - which allows us to move faster since we can spend less time trying to fill these values

The following code replicates this process, if you want to replicate the dataset to work with it on your own:

import pandas as pd <br>
loans_2007 = pd.read_csv('LoanStats3a.csv', skiprows=1) <br>
half_count = len(loans_2007) / 2 <br>
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1) <br>
loans_2007 = loans_2007.drop(['desc', 'url'],axis=1) <br>
loans_2007.to_csv('loans_2007.csv', index=False)

We named the filtered dataset loans_2007.csv instead in case we want to explore the raw dataset (LoanStats3a.csv) without mixing up the two. First things first, let's read in the dataset into a Dataframe so we can start to explore the data and explore the remaining features.

Task(s):
- Read loans_2007.csv into a DataFrame named loans_2007 and use the print function to display the first row of the Dataframe.

- Use the print function to:

    - display the first row of loans_2007 and
    - the number of columns in loans_2007.

In [1]:
# Solution(s):
import pandas as pd
loans_2007 = pd.read_csv('C:/Users/dhagi/Documents/Coursework/Dataquest/ML_Lending_Club/loans_2007.csv')
print(loans_2007.head())
print(loans_2007.shape)
print(len(loans_2007.columns))

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   
1  1077430  1314167.0     2500.0       2500.0           2500.0   60 months   
2  1077175  1313524.0     2400.0       2400.0           2400.0   36 months   
3  1076863  1277178.0    10000.0      10000.0          10000.0   36 months   
4  1075358  1311748.0     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade    ...    last_pymnt_amnt  \
0   10.65%       162.87     B        B2    ...             171.62   
1   15.27%        59.83     C        C4    ...             119.66   
2   15.96%        84.33     C        C5    ...             649.91   
3   13.49%       339.31     C        C1    ...             357.48   
4   12.69%        67.79     B        B5    ...              67.79   

  last_credit_pull_d collections_12_mths_ex_med  policy_code application_type  \
0           Jun-2016               

C:\Users\dhagi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Dropping Unnecessary Columns

The Dataframe contains many columns and can be cumbersome to try to explore all at once. Let's break up the columns into 3 groups of 18 columns and use the data dictionary to become familiar with what each column represents. As you understand each feature, you want to pay attention to any features that:

- leak information from the future (after the loan has already been funded)
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
- formatted poorly and need to be cleaned up
- require more data or a lot of processing to turn into a useful feature
- contain redundant information

We need to especially pay attention to data leakage, since it can cause our model to overfit. This is because the model would be using data about the target column that wouldn't be available when we're using the model on future loans. We encourage you to spend as much time as you need to understand each column, because a poor understanding could cause you to make mistakes in the data analysis and modeling process. As you go through the dictionary, keep in mind that we need to select one of the columns as the target column we want to use when we move on to the machine learning phase.

In this screen and the next few screens, let's focus on just columns that we need to remove from consideration. Then, we can circle back and further dissect the columns we decided to keep.

After analyzing each column, we can conclude that the following features need to be removed:

- id: randomly generated field by Lending Club for unique identification purposes only
- member_id: also a randomly generated field by Lending Club for unique identification purposes only
- funded_amnt: leaks data from the future (after the loan is already started to be funded)
- funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
- grade: contains redundant information as the interest rate column (int_rate)
- sub_grade: also contains redundant information as the interest rate column (int_rate)
- emp_title: requires other data and a lot of processing to potentially be useful
- issue_d: leaks data from the future (after the loan is already completed funded)

Recall that Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the grade and sub_grade values are categorical, the int_rate column contains continuous values, which are better suited for machine learning.

Let's now drop these columns from the Dataframe before moving onto the next group of columns.

Task(s):

Use the Dataframe method drop to remove the following columns from the loans_2007 Dataframe:

- id
- member_id
- funded_amnt
- funded_amnt_inv
- grade
- sub_grade
- emp_title
- issue_d

In [2]:
# Solution(s):

loans_2007 = loans_2007.drop(['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 'issue_d'], axis = 1)

### Removing More Columns

Looking at the next group of columns from the data dictionary, we need to drop the following columns:

- zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- out_prncp: leaks data from the future, (after the loan already started to be paid off)
- out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)

The out_prncp and out_prncp_inv both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. These 2 columns as well as the total_pymnt column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.

Let's go ahead and remove these columns from the Dataframe.

Task(s):

Use the Dataframe method drop to remove the following columns from the loans_2007 Dataframe:

- zip_code
- out_prncp
- out_prncp_inv
- total_pymnt
- total_pymnt_inv
- total_rec_prncp

In [3]:
# Solution(s):

loans_2007 = loans_2007.drop(['zip_code','out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp'], axis = 1)

### Dropping Last Set of Columns

In the last group of columns, we need to drop the following columns:

- total_rec_int: leaks data from the future, (after the loan already started to be paid off),
- total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
- recoveries: also leaks data from the future, (after the loan already started to be paid off),
- collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

Task(s):

Use the Dataframe method drop to remove the following columns from the loans_2007 Dataframe:

- total_rec_int
- total_rec_late_fee
- recoveries
- collection_recovery_fee
- last_pymnt_d
- last_pymnt_amnt

Use the print function to:

- display the first row of loans_2007 and
- the number of columns in loans_2007.

In [4]:
# Solution(s):

loans_2007 = loans_2007.drop(['total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt'], axis=1)

print(loans_2007.head())
print(loans_2007.shape)

   loan_amnt        term int_rate  installment emp_length home_ownership  \
0     5000.0   36 months   10.65%       162.87  10+ years           RENT   
1     2500.0   60 months   15.27%        59.83   < 1 year           RENT   
2     2400.0   36 months   15.96%        84.33  10+ years           RENT   
3    10000.0   36 months   13.49%       339.31  10+ years           RENT   
4     3000.0   60 months   12.69%        67.79     1 year           RENT   

   annual_inc verification_status  loan_status pymnt_plan    ...      \
0     24000.0            Verified   Fully Paid          n    ...       
1     30000.0     Source Verified  Charged Off          n    ...       
2     12252.0        Not Verified   Fully Paid          n    ...       
3     49200.0     Source Verified   Fully Paid          n    ...       
4     80000.0     Source Verified      Current          n    ...       

  initial_list_status last_credit_pull_d collections_12_mths_ex_med  \
0                   f           Jun-201

### Choosing Target Column

Just by becoming familiar with the columns in the dataset, we were able to reduce the number of columns from 52 to 32 columns. We now need to decide on a target column that we want to use for modeling.

We should use the loan_status column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model. Let's explore the different values in this column and come up with a strategy for converting the values in this column.

Task(s):

- Use the Series method value_counts to return the frequency of the unique values in the loan_status column.
- Display the frequency of each unique value using the print function.

In [5]:
# Solution(s):

ls = loans_2007['loan_status'].value_counts()
print(ls)

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


### Analyzing 'loan_status' Column

There are 8 different possible values for the loan_status column. You can read about most of the different loan statuses on the Lending Clube webste. The 2 values that start with "Does not meet the credit policy" aren't explained unfortunately. A quick Google search takes us to explanations from the lending community here and here.

We've compiled the explanation for each column as well as the counts in the Dataframe in the following table:

- Loan Status |	Count |	Meaning
- Fully Paid | 33136 | Loan has been fully paid off.
- Charged Off | 5634 | Loan for which there is no longer a reasonable expectation of further payments.
- Does not meet the credit policy. Status:Fully Paid | 1988 | While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
- Does not meet the credit policy. Status:Charged Off | 761 | While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
- In Grace Period | 20 | The loan is past due but still in the grace period of 15 days.
- Late (16-30 days) | 8 | Loan hasn't been paid in 16 to 30 days (late on the current payment).
- Late (31-120 days) | 24 | Loan hasn't been paid in 31 to 120 days (late on the current payment).
- Current | 961 | Loan is up to date on current payments.
- Default | 3 | Loan is defaulted on and no payment has been made for more than 121 days.

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. You can read about the difference here.

Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case. While there are a few different ways to transform all of the values in a column, we'll use the Dataframe method replace. According to the documentation, we can pass the replace method a nested mapping dictionary in the following format:


mapping_dict = { <br>
    "date": {<br>
        "january": 1,<br>
        "february": 2,<br>
        "march": 3<br>
    }<br>
}<br>
df = df.replace(mapping_dict)

Lastly, one thing we need to keep in mind is the class imbalance between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes. There are a few different ways to tackle this class imbalance, which we'll explore later.

Task(s):

- Remove all rows from loans_2007 that contain values other than Fully Paid or Charged Off for the loan_status column.
- Use the Dataframe method replace to replace:
    - Fully Paid with 1
    - Charged Off with 0

In [6]:
# Solution(s):

loans_2007 = loans_2007[(loans_2007['loan_status'] =='Fully Paid') | (loans_2007['loan_status']=='Charged Off')]
print(loans_2007['loan_status'].head())
loans_2007['loan_status'] = loans_2007['loan_status'].replace(to_replace = ['Fully Paid','Charged Off'],value = [1,0])

0     Fully Paid
1    Charged Off
2     Fully Paid
3     Fully Paid
5     Fully Paid
Name: loan_status, dtype: object


### Removing Columns without Multiple 

To wrap up this mission, let's look for any columns that contain only one unique value and remove them. These columns won't be useful for the model since they don't add any information to each loan application. In addition, removing these columns will reduce the number of columns we'll need to explore further in the next mission.

We'll need to compute the number of unique values in each column and drop the columns that contain only one unique value. While the Series method unique returns the unique values in a column, it also counts the Pandas missing value object nan as a value:

Following line returns 0 and nan. <br>
unique_values = loans['tax_liens'].unique()

Since we're trying to find columns that contain one true unique value, we should first drop the null values then compute the number of unique values:

non_null = loans_2007['tax_liens'].dropna()<br>
unique_non_null = non_null.unique()<br>
num_true_unique = len(unique_non_null)<br>
non_null = loans_2007['tax_liens'].dropna()<br>
unique_non_null = non_null.unique()<br>
num_true_unique = len(unique_non_null)

Task(s):
- Remove any columns from loans_2007 that contain only one unique value:
    - Create an empty list, drop_columns to keep track of which columns you want to drop
    - For each column:
        - Use the Series method dropna to remove any null values and then use the Series method unique to return the set of non-null unique values
        - Use the len() function to return the number of values in that set
        - Append the column to drop_columns if it contains only 1 unique value
    - Use the Dataframe method drop to remove the columns in drop_columns from loans_2007
- Use the print function to display drop_columns so we know which ones were removed

In [7]:
# Solution(s):
drop_columns = []
orig_columns = loans_2007.columns
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis = 1)
print(drop_columns)
print(loans_2007.columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']
Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'last_credit_pull_d',
       'pub_rec_bankruptcies'],
      dtype='object')


### Data Cleaning Conclusion

It looks we we were able to remove 9 more columns since they only contained 1 unique value.

In this mission, we started to become familiar with the columns in the dataset and removed many columns that aren't useful for modeling. We also selected our target column and decided to focus our modeling efforts on binary classification. In the next mission, we'll explore the individual features in greater depth and work towards training our first machine learning model.

### Data Cleaning Recap and Null Value Review

In the past mission, you removed all of the columns that contained redundant information, weren't useful for modeling, required too much processing to make useful, or leaked information from the future. In this mission, we'll prepare the data for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns we encounter throughout this process.

This is because the mathematics underlying most machine learning models assumes that the data is numerical and contains no missing values. To reinforce this requirement, scikit-learn will return an error if you try to train a model using data that contain missing values or non-numeric values when working with models like linear regression and logistic regression.

Let's start by computing the number of missing values and come up with a strategy for handling them. Then, we'll focus on the categorical columns.

We can return the number of missing values across the Dataframe by:

- first using the Pandas Dataframe method isnull to return a Dataframe containing Boolean values:
    - True if the original value is null,
    - False if the original value isn't null.
- then using the Pandas Dataframe method sum to calculate the number of null values in each column.

null_counts = df.isnull().sum()

Task(s):

- Use the isnull and sum methods to return the number of null values in each column. Assign the resulting Series object to null_counts.
- Use the print function to display null_counts.

In [9]:
# Solution(s):

# Updating dataframe name for consistency with guided project
loans = loans_2007

null_counts = loans.isnull().sum()
print(null_counts)

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


### Removing Columns with Large Number of Null Values

While most of the columns have 0 missing values, 2 columns have 50 or less rows with missing values, and 1 column, pub_rec_bankruptcies, contains 697 rows with missing values. Let's remove columns entirely where more than 1% of the rows for that column contain a null value. In addition, we'll remove the remaining rows containing null values.

This means that we'll keep the following columns and just remove rows containing missing values for them:

- title
- revol_util
- last_credit_pull_d

and drop the pub_rec_bankruptcies column entirely since more than 1% of the rows have a missing value for this column.

Let's use the strategy of removing the pub_rec_bankruptcies column first then removing all rows containing any missing values at all to cover both of these cases. This way, we only remove the rows containing missing values for the title and revol_util columns but not the pub_rec_bankruptcies column.

Task(s):
- Use the drop method to remove the pub_rec_bankruptcies column from loans.
- Use the dropna method to remove all rows from loans containing any missing values.
- Use the dtypes attribute followed by the value_counts() method to return the counts for each column data type. Use the print function to display these counts.

In [10]:
# Solution(s):

loans = loans.drop(['pub_rec_bankruptcies'], axis = 1)

loans = loans.dropna()

print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


### Keeping only Object Columns

While the numerical columns can be used natively with scikit-learn, the object columns that contain text need to be converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore them in more depth. You can use the Dataframe method select_dtypes to select only the columns of a certain data type:

float_df = df.select_dtypes(include=['float'])

Let's select just the object columns then display a sample row to get a better sense of how the values in each column are formatted.

Task(s):

- Use the Dataframe method select_dtypes to select only the columns of object type from loans and assign the resulting Dataframe object_columns_df.
- Display the first row in object_columns_df using the print function.

In [11]:
# Solution(s):
object_columns_df = loans.select_dtypes(include=['object'])
print(object_columns_df.head())

         term int_rate emp_length home_ownership verification_status  \
0   36 months   10.65%  10+ years           RENT            Verified   
1   60 months   15.27%   < 1 year           RENT     Source Verified   
2   36 months   15.96%  10+ years           RENT        Not Verified   
3   36 months   13.49%  10+ years           RENT     Source Verified   
5   36 months    7.90%    3 years           RENT     Source Verified   

          purpose                                  title addr_state  \
0     credit_card                               Computer         AZ   
1             car                                   bike         GA   
2  small_business                   real estate business         IL   
3           other                               personel         CA   
5         wedding  My wedding loan I promise to pay back         AZ   

  earliest_cr_line revol_util last_credit_pull_d  
0         Jan-1985      83.7%           Jun-2016  
1         Apr-1999       9.4%         

### Reviewing Feature Columns
Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

- home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
- verification_status: indicates if income was verified by Lending Club,
- emp_length: number of years the borrower was employed upon time of application,
- term: number of payments on the loan, either 36 or 60,
- addr_state: borrower's state of residence,
- purpose: a category provided by the borrower for the loan request,
- title: loan title provided the borrower,

There are also some columns that represent numeric values, that need to be converted:

- int_rate: interest rate of the loan in %,
- revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit, read more here.

Based on the first row's values for purpose and title, it seems like these columns could reflect the same information. Let's explore the unique value counts separately to confirm if this is true.
Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

- earliest_cr_line: The month the borrower's earliest reported credit line was opened,
- last_credit_pull_d: The most recent month Lending Club pulled credit for this loan.

Since these date features require some feature engineering for modeling purposes, let's remove these date columns from the Dataframe.

###  Reviewing Unique Values in Categorical Columns
Let's explore the unique value counts of the columnns that seem like they contain categorical values.

Task(s):
- Display the unique value counts for the following columns: home_ownership, verification_status, emp_length, term, addr_state columns:
    - Store these column names in a list named cols.
    - Use a for loop to iterate over cols:
        - Use the print function combined with the Series method value_counts to display each column's unique value counts.

In [12]:
# Solution(s):
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
LA     420
AL     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
AK      76
WY      76
SD      60
VT  

### Evaluate Purpose and Title Columns

The home_ownership, verification_status, emp_length, term, and addr_state columns all contain multiple discrete values. We should clean the emp_length column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

First, let's look at the unique value counts for the purpose and title columns to understand which column we want to keep.

Task(s):
- Use the value_counts method and the print function to display the unique values in the following columns:
    - purpose
    - title

In [13]:
# Solution(s):
print(loans["purpose"].value_counts())
print(loans["title"].value_counts())

debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64
Debt Consolidation                                 2068
Debt Consolidation Loan                            1599
Personal Loan                                       624
Consolidation                                       488
debt consolidation                                  466
Credit Card Consolidation                           345
Home Improvement                                    336
Debt consolidation                                  314
Small Business Loan                                 298
Credit Card Loan                                    294
Personal            

### Cleaning Feature Columns
The home_ownership, verification_status, emp_length, and term columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

We can use the following mapping to clean the emp_length column:
- "10+ years": 10
- "9 years": 9
- "8 years": 8
- "7 years": 7
- "6 years": 6
- "5 years": 5
- "4 years": 4
- "3 years": 3
- "2 years": 2
- "1 year": 1
- "< 1 year": 0
- "n/a": 0

We erred on the side of being conservative with the 10+ years, < 1 year and n/a mappings. We assume that people who may have been working more than 10 years have only really worked for 10 years. We also assume that people who've worked less than a year or if the information is not available that they've worked for 0. This is a general heuristic but it's not perfect.

Lastly, the addr_state column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.

Task(s):
- Remove the last_credit_pull_d, addr_state, title, and earliest_cr_line columns from loans.
- Convert the int_rate and revol_util columns to float columns by:
    - Using the str acessor followed by the rstrip string method to strip the right trailing percent sign (%):
        - loans['int_rate'].str.rstrip('%') returns a new Series with % stripped from the right side of each value.
    - On the resulting Series object, use the astype method to convert to the float type.
    - Assign the new Series of float values back to the respective columns in the Dataframe.
- Use the replace method to clean the emp_length column.

In [14]:
# Solution(s):
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype('float')
loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype('float')
loans = loans.replace(mapping_dict)

### Dummy Encoding

Let's now encode the home_ownership, verification_status, purpose, and term columns as dummy variables so we can use them in our model. We first need to use the Pandas get_dummies method to return a new Dataframe containing a new column for each dummy variable:


Returns a new Dataframe containing 1 column for each dummy variable.

    dummy_df = pd.get_dummies(loans[["term", "verification_status"]])

We can then use the concat method to add these dummy columns back to the original Dataframe:

    loans = pd.concat([loans, dummy_df], axis=1)

and then drop the original columns entirely using the drop method:

    loans = loans.drop(["verification_status", "term"], axis=1)

Task(s):
- Encode the home_ownership, verification_status, purpose, and term columns as integer values:
    - Use the get_dummies function to return a Dataframe containing the dummy columns.
    - Use the concat method to add these dummy columns back to loans.
    - Remove the original, non-dummy columns (home_ownership, verification_status, purpose, and term) from loans.

In [15]:
# Solution(s):
dummy_df = pd.get_dummies(loans[['home_ownership', 'verification_status', 'purpose', 'term']])
loans = pd.concat([loans,dummy_df], axis = 1)
loans = loans.drop(['home_ownership', 'verification_status', 'purpose', 'term'],axis = 1)

### Data Preparation Conclusion
In this mission, we performed the last amount of data preparation necessary to start training machine learning models. We converted all of the columns to numerical values because those are the only type of value scikit-learn can work with. In the next mission, we'll experiment with training models and evaluating accuracy using cross-validation.

### Data Cleaning and Preparation Recap
We spent the last 2 missions cleaning and preparing a dataset that contains data on loans made to members of Lending Club. Our eventual goal is to generate features from the data, which can feed into a machine learning algorithm. The algorithm will make predictions about whether or not a loan will be paid off on time, which is contained in the loan_status column of the clean dataset.

As we prepared the data, we removed columns that had data leakage issues, contained redundant information, or required additional processing to turn into useful features. We cleaned features that had formatting issues, and converted categorical columns to dummy variables. 

In the last mission, we noticed that there's a class imbalance in our target column, loan_status. There are about 6 times as many loans that were paid off on time (positive case, label of 1) than those that weren't (negative case, label of 0). Imbalances can cause issues with many machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data. Because of its potential to cause issues, we need to keep the class imbalance in mind as we build machine learning models.

After all of our data cleaning in the past two missions, we ended up with the csv file called cleaned_loans_2007.csv. Let's read this file into a Dataframe and view a summary of the work we did.

Task(s):
- Read cleaned_loans_2007.csv into a Dataframe named loans.
- Use the info() method and the print function to display a summary of the dataset.

In [16]:
# Solution(s):
import pandas as pd
# loans = pd.read_csv('cleaned_loans_2007.csv') - Commenting as data is already read in
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37675 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              37675 non-null float64
int_rate                               37675 non-null float64
installment                            37675 non-null float64
emp_length                             37675 non-null int64
annual_inc                             37675 non-null float64
loan_status                            37675 non-null int64
dti                                    37675 non-null float64
delinq_2yrs                            37675 non-null float64
inq_last_6mths                         37675 non-null float64
open_acc                               37675 non-null float64
pub_rec                                37675 non-null float64
revol_bal                              37675 non-null float64
revol_util                             37675 non-null float64
total_acc                              37675 non-null float64
home_ownership_MORTGAGE    

### Understanding Error Metrics

Before we dive into predicting loan_status with machine learning, let's go back to our first steps when we started cleaning the Lending Club dataset. You may recall the original question we wanted to answer:
    
- Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

We established that this is a binary classification problem in the first mission of this course, and we converted the loan_status column to 0s and 1s as a result. Before diving in and selecting an algorithm to apply to the data, we should select an error metric. 

An error metric will help us figure out when our model is performing well, and when it's performing poorly. To tie error metrics all the way back to the original question we wanted to answer, let's say we're using a machine learning model to predict whether or not we should fund a loan on the Lending Club platform. Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money or lose us money.

In this case, we're primarily concerned with false positives and false negatives. Both of these are different types of misclassifications. With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off. 

In the loan_status and prediction columns, a 0 means that the loan wouldn't be paid off on time, and a 1 means that it would.

Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk, and avoid false positives as much as possible. They'd be more okay with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).

Let's calculate false positives and true positives in Python. We can use multiple conditionals, separated by a & to select items in a NumPy array that meet certain conditions. For instance, if we had an array called predictions, we could select items in predictions that equal 1 and where items in loans["loan_status"] in the same position also equal 1 using this:


    filter = (predictions == 1) & (loans["loan_status"] == 1)
    predictions[tp_filter]

The above code will give us all the items in predictions that are true positives -- where we predicted that the loan would be paid off on time, and it was actually paid off on time. By using the len function to find the number of items, we can find the number of true positives.

### Making Predictions with Logistic Regression
Now that we've setup error metrics, we can move on to making predictions using a machine learning algorithm.

As we saw in the first screen of the mission, our cleaned dataset contains 41 columns, all of which are either the int64 or the float64 data type. There aren't any null values in any of the columns. This means that we can now apply any machine learning algorithm to our dataset. Most algorithms can't deal with non-numeric or missing values, which is why we had to do so much data cleaning.

In order to fit the machine learning models, we'll use the Scikit-learn library. Although we've built our own implementations of algorithms in earlier missions, it's easier and faster to use algorithms that someone else has already written and tuned for high performance.

A good first algorithm to apply to binary classification problems is logistic regression, for the following reasons:
- it's quick to train and we can iterate more quickly,
- it's less prone to overfitting than more complex models like decision trees,
- it's easy to interpret.

Task(s):

Now that we've setup error metrics, we can move on to making predictions using a machine learning algorithm.

As we saw in the first screen of the mission, our cleaned dataset contains 41 columns, all of which are either the int64 or the float64 data type. There aren't any null values in any of the columns. This means that we can now apply any machine learning algorithm to our dataset. Most algorithms can't deal with non-numeric or missing values, which is why we had to do so much data cleaning.

In order to fit the machine learning models, we'll use the Scikit-learn library. Although we've built our own implementations of algorithms in earlier missions, it's easier and faster to use algorithms that someone else has already written and tuned for high performance.

A good first algorithm to apply to binary classification problems is logistic regression, for the following reasons:
- it's quick to train and we can iterate more quickly,
- it's less prone to overfitting than more complex models like decision trees,
- it's easy to interpret.

In [18]:
# Solution(s):
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

cols = loans.columns
train_cols = cols.drop('loan_status')
features = loans[train_cols]
target = loans['loan_status']

lr.fit(features, target)
predictions = lr.predict(features)

### Using K-Folds Cross Validation
While we generated predictions in the last screen, those predictions were overfit. They were overfit because we generated predictions using the same data that we trained our model on. When we use this to evaluate error, we get an unrealistically high depiction of how accurate the algorithm is, because it already "knows" the correct answers. This is like asking someone to memorize a bunch of physics equations, then asking them to plug numbers into the equations. They can tell you the right answer, but they can't explain a concept that they haven't already memorized an equation for.

In order to get a realistic depiction of the accuracy of the model, let's perform k-fold cross validation. We can use the cross_val_predict() function from the sklearn.model_selection package. Here's what the workflow looks like:

    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_predict

    #Instantiate model object.
    lr = LogisticRegression()
    #Make predictions using 3-fold cross-validation.
    cross_val_predict(lr, features, target, cv=3)

Once we have cross validated predictions, we can compute true positive rate and false positive rate.

Task(s):
- Generate cross validated predictions for features.
    - Call cross_val_predict using lr, features, and target.
    - Set the kf parameter to 3, so that 3-fold cross validation is performed.
    - Assign the predictions to predictions.
- Use the Series class to convert predictions to a pandas Series object.
- Compute true positive rate and false positive rate.
    - Assign true positive rate to tpr.
    - Assign false positive rate to fpr.
- Display fpr and tpr to evaluate them.

In [20]:
# Solution(s):
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression()
predictions = cross_val_predict(lr,features,target, cv = 3)
predictions = pd.Series(predictions)

fp_filter = (predictions ==1) & (loans['loan_status'] == 0)
fp = len(predictions[fp_filter])

tp_filter = (predictions==1) & (loans['loan_status'] == 1)
tp = len(predictions[tp_filter])

fn_filter = (predictions==0) & (loans['loan_status'] == 1)
fn = len(predictions[fn_filter])

tn_filter = (predictions == 0) & (loans['loan_status'] == 0)
tn = len(predictions[tn_filter])

tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print(tpr)
print(fpr)

0.9984268484530676
0.9986179664363277


### Correcting for Unbalanced Classes
As you can see from the last screen, our fpr and tpr are around what we'd expect if the model was predicting all ones.

Unfortunately, even through we're not using accuracy as an error metric, the classifier is, and it isn't accounting for the imbalance in the classes. There are a few ways to get a classifier to correct for imbalanced classes. The two main ways are:
- Use oversampling and undersampling to ensure that the classifier gets input that has a balanced number of each class.
- Tell the classifier to penalize misclassifications of the less prevalent class more than the other class.
We'll look into oversampling and undersampling first. They involve taking a sample that contains equal numbers of rows where loan_status is 0, and where loan_status is 1. This way, the classifier is forced to make actual predictions, since predicting all 1s or all 0s will only result in 50% accuracy at most.

The downside of this technique is that since it has to preserve an equal ratio, you have to either:
- Throw out many rows of data. If we wanted equal numbers of rows where loan_status is 0 and where loan_status is 1, one way we could do that is to delete rows where loan_status is 1.
- Copy rows multiple times. One way to equalize the 0s and 1s is to copy rows where loan_status is 0.
- Generate fake data. One way to equalize the 0s and 1s is to generate new rows where loan_status is 0.
Unfortunately, none of these techniques are especially easy. The second method we mentioned earlier, telling the classifier to penalize certain rows more, is actually much easier to implement using scikit-learn.

We can do this by setting the class_weight parameter to balanced when creating the LogisticRegression instance. This tells scikit-learn to penalize the misclassification of the minority class during the training process. The penalty means that the logistic regression classifier pays more attention to correctly classifying rows where loan_status is 0. This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.

By setting the class_weight parameter to balanced, the penalty is set to be inversely proportional to the class frequencies. You can read more about the parameter here. This would mean that for the classifier, correctly classifying a row where loan_status is 0 is 6 times more important than correctly classifying a row where loan_status is 1.

We can repeat the cross validation procedure we performed in the last screen, but with the class_weight parameter set to balanced.

Task(s):
- Create a LogisticRegression instance.
    - Remember to set class_weight to balanced.
    - Assign the instance to lr.
- Generate cross validated predictions for features.
    - Call cross_val_predict() using lr, features, and target.
    - Assign the predictions to predictions.
- Use the Series class to convert predictions to a Pandas Series, as we did in the last screen.
    - Converting to Series objects letse us take advantage of boolean filtering and arithmetic operations from pandas.
- Compute true positive rate and false positive rate.
    - Assign true positive rate to tpr.
    - Assign false positive rate to fpr.
- Print out fpr and tpr to evaluate them.

In [22]:
# Solution(s):
lr = LogisticRegression(class_weight = 'balanced')

predictions = cross_val_predict(lr, features, target, cv = 3)
predictions = pd.Series(predictions)

tp_filter = (predictions==1) & (loans['loan_status']==1)
tp = len(loans[tp_filter])

fp_filter = (predictions==1) & (loans['loan_status']==0)
fp = len(loans[fp_filter])

tn_filter = (predictions==0) & (loans['loan_status']==0)
tn = len(loans[tn_filter])

fn_filter = (predictions==0) & (loans['loan_status']==1)
fn = len(loans[fn_filter])

tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.6288345568956476
0.6159921026653504


C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Attempting to Lower False Positives
We significantly improved false positive rate in the last screen by balancing the classes, which reduced true positive rate. Our true positive rate is now around 63%, and our false positive rate is around 62%. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything. However, we'd only ever decide to fund 63% of the total loans (true positive rate), so we'd immediately reject a good amount of loans.

We can try to lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class. While setting class_weight to balanced will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty. In the last screen, the penalty scikit-learn imposed for misclassifying a 0 would have been around 5.89 (since there are 5.89 times as many 1s as 0s).

We can also specify a penalty manually if we want to adjust the rates more. To do this, we need to pass in a dictionary of penalty values to the class_weight parameter:


    penalty = {
        0: 10,
        1: 1
    }
    lr = LogisticRegression(class_weight=penalty)

The above dictionary will impose a penalty of 10 for misclassifying a 0, and a penalty of 1 for misclassifying a 1.

Task(s):
- Modify the code from the last screen to change the class_weight parameter from the string "balanced" to the dictionary:
    penalty = {
        0: 10,
        1: 1
    }
    
- Remember to print out the fpr and tpr values at the end!

In [24]:
# Solution(s):
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight = penalty)

predictions = cross_val_predict(lr, features, target, cv = 3)
predictions = pd.Series(predictions)

tp_filter = (predictions==1) & (loans['loan_status']==1)
tp = len(loans[tp_filter])

fp_filter = (predictions==1) & (loans['loan_status']==0)
fp = len(loans[fp_filter])

tn_filter = (predictions==0) & (loans['loan_status']==0)
tn = len(loans[tn_filter])

fn_filter = (predictions==0) & (loans['loan_status']==1)
fn = len(loans[fn_filter])

tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.2305977975878343
0.22724580454096743


C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Implementing Random Forest

It looks like assigning manual penalties lowered the false positive rate to 23%, and thus lowered our risk. Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

While we could tweak the penalties further, it's best to move to trying a different model right now, for larger potential false positive rate gains. We can always loop back and iterate on the penalties more later.

Let's try a more complex algorithm, random forest. We learned about random forests in a previous mission, and constructed our own model. Random forests are able to work with nonlinear data, and learn complex conditionals. Logistic regressions are only able to work with linear data. Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with loan_status.

We can use the RandomForestClassifer class from scikit-learn to do this.

Task(s):
- Modify the code from the last screen, and swap out the LogisticRegression for a RandomForestClassifer model.
    - Set the value of the keyword argument random_state to 1, so the predictions don't vary due to random chance.
    - Set the value of the keyword argument class_weight to balanced, so we avoid issues with imbalanced classes.
- Remember to print out the fpr and tpr values at the end!

In [25]:
# Solution(s):
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict

penalty = {
    0: 20,
    1: 1
}

lr = RandomForestClassifier(random_state = 1, class_weight = 'balanced')
#lr = RandomForestClassifier(random_state = 1, class_weight = penalty)

predictions = cross_val_predict(lr, features, target, cv = 3)
predictions = pd.Series(predictions)

tp_filter = (predictions==1) & (loans['loan_status']==1)
tp = len(loans[tp_filter])

fp_filter = (predictions==1) & (loans['loan_status']==0)
fp = len(loans[fp_filter])

tn_filter = (predictions==0) & (loans['loan_status']==0)
tn = len(loans[tn_filter])

fn_filter = (predictions==0) & (loans['loan_status']==1)
fn = len(loans[fn_filter])

tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

C:\Users\dhagi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.9630964866282119
0.9634748272458046


C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\dhagi\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Conclusion:
Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

Ultimately, our best model had a false positive rate of 7%, and a true positive rate of 20%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 7% of borrowers defaulting, and that the pool of 20% of borrowers is large enough to make enough interest money to offset the losses.

If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and our model is better than that, although we're excluding more loans than a random strategy would. Given this, there's still quite a bit of room to improve:

- We can tweak the penalties further.
- We can try models other than a random forest and logistic regression.
- We can use some of the columns we discarded to generate better features.
- We can ensemble multiple models to get more accurate predictions.
- We can tune the parameters of the algorithm to achieve higher performance.